In [2]:
import os

In [3]:
%pwd


'f:\\resume\\text_summarizer\\research'

In [4]:
os.chdir('..')
%pwd

'f:\\resume\\text_summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories


In [15]:
class ConfigurationManager:
    def __init__(self):
        config_file_path = CONFIG_FILE_PATH
        params_file_path = PARAMS_FILE_PATH

        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [17]:
import os 
import urllib.request as request
import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file,
            )
            logger.info(f"{filename} downloaded with following info \n {headers}")
        else:
            logger.info(f'file already exists.')
    def extract_zipfile(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as z_ref:
            z_ref.extractall(unzip_path)

In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zipfile()
except Exception as e:
    raise e

[2024-07-28 09:18:53,508: yaml file: config\config.yaml loaded successfully: common: yaml file: config\config.yaml loaded successfully]
[2024-07-28 09:18:53,512: yaml file: params.yaml loaded successfully: common: yaml file: params.yaml loaded successfully]
[2024-07-28 09:18:53,516: created directory at: artifacts: common: created directory at: artifacts]
[2024-07-28 09:18:53,517: created directory at: artifacts/data_ingestion: common: created directory at: artifacts/data_ingestion]
[2024-07-28 09:19:00,315: artifacts/data_ingestion/data.zip downloaded with following info 
 Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2289:0DE8:213942